# 간단한 피처 엔지니어링 + LightGBM 베이스라인 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [2]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [3]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

## 2. 피처 엔지니어링 


### 2-1. 필요 없는 피처 제거 및 필요한 피처 생성 

In [4]:
drop_features = []

In [5]:
# "시술 시기 코드", "특정 시술 유형", "배아 생성 주요 이유" 제거
# "정자 출처", "난자 기증자 나이", "정자 기증자 나이" 제거
# "주요 시술 유형" 생성

cat_features = [ 
    "시술 당시 나이", 
    "시술 유형",
    "주요 시술 유형",
    "배란 유도 유형",
    "난자 출처"
]

len(cat_features)

5

In [6]:
# "총 시술 횟수", "클리닉 내 총 시술 횟수", "총 임신 횟수", "총 출산 횟수" 제거
# "IVF 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수" 제거
# "DI 시술 횟수", "DI 임신 횟수", "DI 출산 횟수" 제거
# "IVF 임신 성공률", "IVF 출산 성공률", "IVF 실패 횟수", "IVF 유산 횟수", "DI 임신 성공률", "DI 출산 성공률", "DI 실패 횟수", "DI 유산 횟수" 생성 

ord_features = [
    "IVF 임신 성공률", 
    "IVF 출산 성공률", 
    "IVF 실패 횟수", 
    "IVF 유산 횟수", 
    "DI 임신 성공률", 
    "DI 출산 성공률", 
    "DI 실패 횟수", 
    "DI 유산 횟수"
]

len(ord_features)

8

In [7]:
# "난자 채취 경과일", "난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일" 제거
# "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "저장된 신선 난자 수", "혼합된 난자 수", "기증자 정자와 혼합된 난자 수" 제거

num_features = [
    "이식된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "배아 이식 경과일",
]

len(num_features)

5

In [8]:
# "남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
# "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증" 제거
# "부부 주 불임 원인", "부부 부 불임 원인" 제거
# "불명확 불임 원인" 제거
# "남성 불임 원인", "여성 불임 원인", "부부 불임 원인" 생성
# "기증 배아 사용 여부" 제거
# "대리모 여부", "PGD 시술 여부", "PGS 시술 여부" 제거

bin_features = [
    '배란 자극 여부',
    '단일 배아 이식 여부',
    '착상 전 유전 진단 사용 여부',
    '남성 불임 원인',
    '여성 불임 원인',
    '부부 불임 원인',
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
]

len(bin_features)

8

In [9]:
# 시술 유형(cat feature) 피처 엔지니어링 

all_data["주요 시술 유형"] = all_data["특정 시술 유형"].apply(lambda x: 
    "ICSI" if "ICSI" in str(x) else 
    "IVF" if "IVF" in str(x) else 
    "IUI" if "IUI" in str(x) else 
    "Other"
)

drop_features = drop_features + ['특정 시술 유형']

In [10]:
# 불임 원인(bin features) 피처 엔지니어링 

infertility_features = ["남성 주 불임 원인", "남성 부 불임 원인", "불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태",
                        "여성 주 불임 원인", "여성 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
                        "부부 주 불임 원인", "부부 부 불임 원인"]

all_data["남성 불임 원인"] = (
    all_data["남성 주 불임 원인"] + 
    all_data["남성 부 불임 원인"] + 
    all_data["불임 원인 - 남성 요인"] + 
    all_data["불임 원인 - 정자 농도"] + 
    all_data["불임 원인 - 정자 면역학적 요인"] + 
    all_data["불임 원인 - 정자 운동성"] + 
    all_data["불임 원인 - 정자 형태"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["여성 불임 원인"] = (
    all_data["여성 주 불임 원인"] + 
    all_data["여성 부 불임 원인"] + 
    all_data["불임 원인 - 난관 질환"] + 
    all_data["불임 원인 - 배란 장애"] + 
    all_data["불임 원인 - 여성 요인"] + 
    all_data["불임 원인 - 자궁경부 문제"] + 
    all_data["불임 원인 - 자궁내막증"]
).apply(lambda x: 1 if x > 0 else 0)

all_data["부부 불임 원인"] = (
    all_data["부부 주 불임 원인"] + 
    all_data["부부 부 불임 원인"]
).apply(lambda x: 1 if x > 0 else 0)

drop_features = drop_features + infertility_features

In [11]:
# 과거 이력(ord feature) 피처 엔지니어링

past_features = ["IVF 시술 횟수", "DI 시술 횟수", "IVF 임신 횟수", "DI 임신 횟수", "IVF 출산 횟수", "DI 출산 횟수"]

for col in past_features:
    all_data[col] = all_data[col].str.replace("회 이상", "").str.replace("회", "").astype(float)

all_data["IVF 임신 성공률"] = all_data["IVF 임신 횟수"] / (all_data["IVF 시술 횟수"] + 1)
all_data["IVF 출산 성공률"] = all_data["IVF 출산 횟수"] / (all_data["IVF 임신 횟수"] + 1)
all_data["IVF 실패 횟수"] = all_data["IVF 시술 횟수"] - all_data["IVF 임신 횟수"]
all_data["IVF 유산 횟수"] = all_data["IVF 임신 횟수"] - all_data["IVF 출산 횟수"]

all_data["DI 임신 성공률"] = all_data["DI 임신 횟수"] / (all_data["DI 시술 횟수"] + 1)
all_data["DI 출산 성공률"] = all_data["DI 출산 횟수"] / (all_data["DI 임신 횟수"] + 1)
all_data["DI 실패 횟수"] = all_data["DI 시술 횟수"] - all_data["DI 임신 횟수"]
all_data["DI 유산 횟수"] = all_data["DI 임신 횟수"] - all_data["DI 출산 횟수"]

drop_features = drop_features + past_features

In [12]:
drop_features = drop_features + ["시술 시기 코드", "배아 생성 주요 이유", "정자 출처", "난자 기증자 나이", "정자 기증자 나이",
                                "총 시술 횟수", "클리닉 내 총 시술 횟수", "총 임신 횟수", "총 출산 횟수",
                                "난자 채취 경과일", "난자 해동 경과일", "난자 혼합 경과일", "배아 해동 경과일", "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "저장된 신선 난자 수", "혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
                                "불명확 불임 원인", "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부"]

In [13]:
extracted_data = all_data.drop(columns=drop_features, errors='ignore')

In [14]:
extracted_features = extracted_data.columns
extracted_features

Index(['시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '배란 자극 여부', '배란 유도 유형',
       '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '이식된 배아 수',
       '해동 난자 수', '수집된 신선 난자 수', '파트너 정자와 혼합된 난자 수', '난자 출처', '동결 배아 사용 여부',
       '신선 배아 사용 여부', '배아 이식 경과일', '주요 시술 유형', '남성 불임 원인', '여성 불임 원인',
       '부부 불임 원인', 'IVF 임신 성공률', 'IVF 출산 성공률', 'IVF 실패 횟수', 'IVF 유산 횟수',
       'DI 임신 성공률', 'DI 출산 성공률', 'DI 실패 횟수', 'DI 유산 횟수'],
      dtype='object')

In [15]:
len(extracted_features)

28

### 2-2. Categorical 피처: 원-핫 인코딩

In [16]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(extracted_data[cat_features])

encoded_cat_matrix

<346418x20 sparse matrix of type '<class 'numpy.float64'>'
	with 1732090 stored elements in Compressed Sparse Row format>

In [17]:
remaining_features = list(set(extracted_features) - set(cat_features))

In [18]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(extracted_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [19]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

## 4. 모델 훈련

In [20]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [21]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.2, 
                                                      random_state=0)

# 베이지안 최적화용 데이터셋
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [22]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'num_leaves': (30, 40),
                'lambda_l1': (0.7, 0.9),
                'lambda_l2': (0.9, 1),
                'feature_fraction': (0.6, 0.7),
                'bagging_fraction': (0.6, 0.9),
                'min_child_samples': (6, 10),
                'min_child_weight': (10, 40)}

# 값이 고정된 하이퍼파라미터
fixed_params = {'objective': 'binary',
                'learning_rate': 0.005,
                'bagging_freq': 1,
                'force_row_wise': True,
                'random_state': 1991}

In [23]:
from sklearn.metrics import roc_auc_score
from lightgbm import early_stopping

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터 
    params = {'num_leaves': int(round(num_leaves)),
              'lambda_l1': lambda_l1,
              'lambda_l2': lambda_l2,
              'feature_fraction': feature_fraction,
              'bagging_fraction': bagging_fraction,
              'min_child_samples': int(round(min_child_samples)),
              'min_child_weight': min_child_weight,
              'feature_pre_filter': False}
    # 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)
    
    print('하이퍼파라미터:', params)    
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=params, 
                           train_set=bayes_dtrain,
                           num_boost_round=2500,
                           valid_sets=bayes_dvalid,
                           callbacks=[early_stopping(stopping_rounds=100)])
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid) 
    # roc-auc 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'roc-auc : {roc_auc}\n')
    
    return roc_auc

In [24]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function,      # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)

In [25]:
# 베이지안 최적화 수행
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 52867, number of negative: 152213
[LightGBM] [Info] Total Bins 300
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.257787 -> initscore=-1.057502
[LightGBM] [Info] Start training from score -1.057502
Training until validation scores don't improve for 100 rounds
Early 

In [26]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.6098490719233953,
 'feature_fraction': 0.6688125838196927,
 'lambda_l1': 0.8699553079100838,
 'lambda_l2': 0.98944104904292,
 'min_child_samples': 6.217770344053975,
 'min_child_weight': 19.157977738117577,
 'num_leaves': 30.03686245884437}

In [27]:
# 정수형 하이퍼파라미터 변환
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [28]:
# 값이 고정된 하이퍼파라미터 추가
max_params.update(fixed_params)

In [29]:
max_params

{'bagging_fraction': 0.6098490719233953,
 'feature_fraction': 0.6688125838196927,
 'lambda_l1': 0.8699553079100838,
 'lambda_l2': 0.98944104904292,
 'min_child_samples': 6,
 'min_child_weight': 19.157977738117577,
 'num_leaves': 30,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [30]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) 
# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) 

# OOF 방식으로 모델 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] # 훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터셋
                          
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params,    # 최적 하이퍼파라미터
                          train_set=dtrain,     # 훈련 데이터셋
                          num_boost_round=2500, # 부스팅 반복 횟수
                          valid_sets=dvalid,    # 성능 평가용 검증 데이터셋
                          callbacks=[early_stopping(stopping_rounds=100)])
    
    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측 
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 ROC-AUC
    roc_auc = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} roc-auc : {roc_auc}\n')

######################################## 폴드 1 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Total Bins 296
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1847]	valid_0's binary_logloss: 0.492185
폴드 1 roc-auc : 0.7292721001057916

######################################## 폴드 2 / 폴드 5 ########################################
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Total Bins 294
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054554
[LightGBM] [Info] Start tr

In [31]:
print('OOF 검증 데이터 roc-auc :', roc_auc_score(y, oof_val_preds))

OOF 검증 데이터 roc-auc : 0.7297187513266823


In [32]:
submission['임신 성공 여부'] = oof_test_preds
submission.to_csv('submission.csv')